In [1]:
from __future__ import print_function

import os
import sys
import numpy as np

from keras.optimizers import SGD
from keras.callbacks import CSVLogger, ModelCheckpoint

sys.path.append(os.path.join(os.getcwd(), os.pardir))

import config

from utils.dataset.data_generator import DataGenerator
from models.cnn3 import cnn

Using Theano backend.
Using gpu device 0: GeForce GTX 680 (CNMeM is disabled, cuDNN 5005)


In [2]:
lr=0.001
n_epochs=500
batch_size=32
input_shape=(140, 140, 1)

name = 'cnn_140_edges_dil_ero_lr_%f_nesterov_r' % lr

In [3]:
print('loading model...')
model = cnn(input_shape=input_shape)
model.summary()

optimizer = SGD(lr=lr, clipnorm=1., clipvalue=0.5, nesterov=True)

print('compiling model...')
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
print('done.')

csv_logger = CSVLogger('%s_training.log' % name)
best_model_checkpointer = ModelCheckpoint(filepath=("./%s_training_weights_best.hdf5" % name), verbose=1,
                                          save_best_only=True)

current_model_checkpointer = ModelCheckpoint(filepath=("./%s_training_weights_current.hdf5" % name), verbose=0)

loading model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 140, 140, 1)   0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 140, 140, 128) 6400        input_1[0][0]                    
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 140, 140, 128) 0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 70, 70, 128)   0           activation_1[0][0]               
__________________________________________________________________________

In [4]:
# model.load_weights('cnn_140_grey_lr_0.070000_nesterov_training_weights_best.hdf5')
# model.load_weights('cnn_140_grey_lr_0.035000_nesterov_1_training_weights_best.hdf5')
# model.load_weights('cnn_140_grey_lr_0.010000_nesterov_2_training_weights_best.hdf5')

In [5]:
print('Initializing data generators...')
train_data_gen = DataGenerator(dataset_file=config.train_data_file, batch_size=batch_size, preprocessing='edges_dil_ero')
validation_data_gen = DataGenerator(dataset_file=config.validation_data_file, batch_size=batch_size, preprocessing='edges_dil_ero')
test_data_gen = DataGenerator(dataset_file=config.test_data_file, batch_size=batch_size, preprocessing='edges_dil_ero')
print('done.')

Initializing data generators...
done.


In [ ]:
print('Fitting model...')
history = model.fit_generator(train_data_gen,
                              nb_epoch=n_epochs,
                              samples_per_epoch=train_data_gen.n_batches * batch_size,
                              validation_data=validation_data_gen,
                              nb_val_samples=validation_data_gen.n_samples,
                              verbose=1,
                              callbacks=[csv_logger, best_model_checkpointer, current_model_checkpointer])
print('done.')

Fitting model...
Epoch 1/500
 1536/10496 [===>..........................] - ETA: 299s - loss: 0.6926 - acc: 0.6136

In [ ]:
### print('Evaluating model...')
score = model.evaluate_generator(test_data_gen, val_samples=test_data_gen.n_samples)
print('done.')

print('Test score:', score[0])
print('Test accuracy:', score[1])